In [85]:
from typing import List
import sys
sys.setrecursionlimit(10**6)

class Solution:
    # minimax without alpha-beta pruning
    # O(2^n) time complexity
    def stoneGameMM(self, piles: List[int]) -> bool:
        def minimax(i, j, turn) -> int:
            if i == j:
                return piles[i] if turn else -piles[i]
            if turn:
                return max(piles[i] + minimax(i+1, j, not turn), piles[j] + minimax(i, j-1, not turn))
            else:
                return min(-piles[i] + minimax(i+1, j, not turn), -piles[j] + minimax(i, j-1, not turn))
            
        return (minimax(0, len(piles)-1, True) > 0)
    
    # minimax with alpha-beta pruning
    # still O(2^n) time complexity
    def stoneGameAB(self, piles: List[int]) -> bool:
        def minimax(i: int, j: int, a: int, b: int, turn: bool) -> int:
            if i == j:
                return piles[i] if turn else -piles[i]
            indices = [(i, piles[i]), (j, piles[j])]
            indices.sort(key=lambda x: -x[1])
            # print(indices)

            if turn:
                max_eval = float('-inf')
                for k, score in indices:
                    eval = score + minimax(i+(k==i), j-(k==j), a, b, not turn)
                    max_eval = max(max_eval, eval)
                    a = max(a, eval)
                    if a >= b:
                        break
                return max_eval
            else:
                min_eval = float('inf')
                for k, score in indices:
                    eval = -score + minimax(i+(k==i), j-(k==j), a, b, not turn)
                    min_eval = min(min_eval, eval)
                    b = min(b, eval)
                    if a >= b:
                        break
                return min_eval
            
        score = minimax(0, len(piles)-1, float('-inf'), float('inf'), True)
        print("score:", score)
        return (score > 0)
    
    # dynamic programming approach
    # the subproblem is to find the maximum score difference between two players
    # O(n^2) time complexity
    # assume that we have an array of length 1, then the maximum score difference is piles[i]
    # if we have an array of length 2 (j = i+1), then the scores are:
    # piles[i] - piles[j] or piles[j] - piles[i]. 
    # The maximum score difference is S = max(piles[i] - piles[j], piles[j] - piles[i])
    # S = max(piles[i] - helper(i+1, j), piles[j] - helper(i, j-1))
    # since each player is attempting to maximize their score, they want to take piles[k] - opponent_score
    # where opponent_score is the score of the other player on the next turn = helper(i+1, j) or helper(i, j-1)
    def stoneGameDP(self, piles: List[int]) -> bool:
        n = len(piles)
        dp = {}
        
        def maximizer(i: int, j: int) -> int:
            if i > j:
                return 0
            if (i, j) in dp:
                return dp[(i, j)]
            
            dp[(i, j)] = max(piles[i] - maximizer(i+1, j), piles[j] - maximizer(i, j-1))
            return dp[(i, j)]
        
        score = maximizer(0, n-1)
        print("score:", score)
        return score > 0
    
    # recursive approach, without memoization
    def stoneGameRec(self, piles: List[int]) -> bool:
        n = len(piles)
        
        def maximizer(i: int, j: int) -> int:
            if i > j:
                return 0
            
            return max(piles[i] - maximizer(i+1, j), piles[j] - maximizer(i, j-1))
        
        score = maximizer(0, n-1)
        print("score:", score)
        return (score > 0)

    # iterative approach using DP
    def stoneGameIter(self, piles: List[int]) -> bool:
        n = len(piles)
        stack = [(0, n-1, True)]  # (i, j, is_maximizer)
        memo = {}  # To store computed results
        
        while stack:
            i, j, is_maximizer = stack[-1]  # Peek at the top of the stack
            
            if (i, j, is_maximizer) in memo:
                stack.pop()  # Remove this state as it's already computed
                continue
            
            if i > j:
                memo[(i, j, is_maximizer)] = 0
                stack.pop()
                continue
            
            left_key = (i+1, j, not is_maximizer)
            right_key = (i, j-1, not is_maximizer)
            
            if left_key not in memo or right_key not in memo:
                # If we don't have results for both subproblems, add them to the stack
                if left_key not in memo:
                    stack.append(left_key)
                if right_key not in memo:
                    stack.append(right_key)
                continue
            
            # If we reach here, we have results for both subproblems
            left_score = piles[i] - memo[left_key]
            right_score = piles[j] - memo[right_key]
            
            result = max(left_score, right_score)
            memo[(i, j, is_maximizer)] = result
            stack.pop()  # Remove this state as we've computed its result
        
        final_score = memo[(0, n-1, True)]
        print("max_score:", final_score)
        return final_score > 0
    
    def stoneGame(self, piles: List[int]) -> bool:
        return self.stoneGameDP(piles)

            
        

In [86]:
import random
import time
import sys
random.seed(0)
sys.setrecursionlimit(10**6)

sol = Solution()
f = sol.stoneGame
# test 1
piles = [5,3,4,5]
print("alice wins:", f(piles), "expected score:", 1, "expected:", True)

# test 2
piles = [3,7,2,3]
print("alice wins:", f(piles), "expected score:", 5, "expected:", True)

# test 3
piles = [1,10,1]
print("alice wins:", f(piles), "expected score:", -8, "expected:", False)

# test 4
n = 10
piles = [random.randint(0, 9) for _ in range(n)]
start = time.time()
print("alice wins:", f(piles), "expected: ?")
print("time:", time.time()-start)

n = 100
piles = [random.randint(0, 9) for _ in range(n)]
start = time.time()
print("alice wins:", f(piles), "expected: ?")
print("time:", time.time()-start)

n = 1000
piles = [random.randint(0, 9) for _ in range(n)]
start = time.time()
print("alice wins:", f(piles), "expected: ?")
print("time:", time.time()-start)

# n = 10000
# piles = [random.randint(0, 9) for _ in range(n)]
# start = time.time()
# print("alice wins:", f(piles), "expected: ?")
# print("time:", time.time()-start)

score: 1
alice wins: True expected score: 1 expected: True
score: 5
alice wins: True expected score: 5 expected: True
score: -8
alice wins: False expected score: -8 expected: False
score: 1
alice wins: True expected: ?
time: 0.0001380443572998047
score: 44
alice wins: True expected: ?
time: 0.004891872406005859
score: 53
alice wins: True expected: ?
time: 0.26718592643737793
